### Read dataset

In [ ]:
import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds
import pandas as pd
import numpy as np
import math
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import glob
import shutil
import keras
from keras.models import Model

from typing import Union
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense, Flatten, Dropout
from keras.models import Sequential

tf.enable_v2_behavior()

from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir(r'/content/drive/My Drive/Colab Notebooks')

Mounted at /content/drive


In [ ]:
annotations = pd.read_csv('annotations.csv')

anno_train = annotations.loc[annotations['Partition'] == 'train']
anno_test = annotations.loc[annotations['Partition'] == 'test']

anno_train.drop(['Partition', 'Number of Annotators who Selected SSA (Out of 7)'], axis=1, inplace=True)
anno_test.drop(['Partition', 'Number of Annotators who Selected SSA (Out of 7)'], axis=1, inplace=True)

anno_train = anno_train.rename(columns={'Majority Vote Label': 'label'})
anno_test = anno_test.rename(columns={'Majority Vote Label': 'label'})

anno_train = anno_train.astype({'label': str})
anno_test = anno_test.astype({'label': str})

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
train_img = anno_train['Image Name'].values
test_img = anno_test['Image Name'].values

# Referenced https://stackoverflow.com/questions/11903037/copy-all-jpg-file-in-a-directory-to-another-directory-in-python
# Referenced https://stackoverflow.com/questions/49237177/copying-multiple-images-in-a-list-from-one-folder-to-another-without-copying-all
src_dir = "images"
train_dir = "train"
test_dir = "test"

for im in train_img:
  shutil.copy(os.path.join(src_dir, im), train_dir)

for im in test_img:
  shutil.copy(os.path.join(src_dir, im), test_dir)

In [ ]:
# Referenced Project A.
# ImageDataGenerator with augmentation
train_datagen = ImageDataGenerator(rescale=1/255.,
shear_range=0.1,
rotation_range=15,
horizontal_flip=True,
vertical_flip=True)

test_datagen = ImageDataGenerator(rescale=1/255.)

# Referenced https://stackoverflow.com/questions/59464409/loading-images-in-keras-for-cnn-from-directory-but-label-in-csv-file
train_generator=train_datagen.flow_from_dataframe(
dataframe=anno_train,
directory="train",
x_col="Image Name",
y_col="label",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(224, 224))

test_generator=test_datagen.flow_from_dataframe(
dataframe=anno_test,
directory="test",
x_col="Image Name",
y_col="label",
batch_size=32,
seed=42,
shuffle=False,
class_mode="categorical",
target_size=(224, 224))

Found 2175 validated image filenames belonging to 2 classes.
Found 977 validated image filenames belonging to 2 classes.


### Test Student

In [ ]:
student = tf.keras.Sequential()
mobile_model= tf.keras.applications.MobileNetV2(include_top=False,
                  input_shape=(224,224,3),
                  pooling='avg',classes=2,
                  weights='imagenet')

mobile_model.trainable = True
for layer in mobile_model.layers:
  layer.trainable = True

student.add(mobile_model)
student.add(Flatten())
student.add(Dense(2))

student.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=['accuracy'])

student.fit_generator(generator=train_generator,
                    steps_per_epoch=train_generator.n // 32,
                    epochs=25,                 
                    shuffle = True,
                    verbose=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/25
67/67 [==============================] - 384s 6s/step - loss: 1.3378 - accuracy: 0.6127
Epoch 2/25
67/67 [==============================] - 33s 485ms/step - loss: 0.5721 - accuracy: 0.7074
Epoch 3/25
67/67 [==============================] - 33s 493ms/step - loss: 0.5633 - accuracy: 0.7513
Epoch 4/25
67/67 [==============================] - 32s 474ms/step - loss: 0.6056 - accuracy: 0.7275
Epoch 5/25
67/67 [==============================] - 32s 482ms/step - loss: 0.5182 - accuracy: 0.7480
Epoch 6/25
67/67 [==============================] - 33s 485ms/step - loss: 0.4928 - accuracy: 0.7825
Epoch 7/25
67/67 [==============================] - 34s 509ms/step - loss: 0.4999 - accuracy: 0.8007
Epoch 8/25
67/67 [==============================] - 32s 476ms/step - loss: 0.4706 - accuracy: 0.8185
Epoch 9/25
67/67 [==============================] - 32s 474ms/step - loss: 0.4073 - accuracy: 0.8371
Epoch 10/25
67/67 [==============================] - 32s 474ms/step - loss: 0.3913 - accuracy

In [6]:
predict = student.predict(test_generator,steps=31)

test_generator.reset()
y_predict = np.argmax(predict, axis=1)
labels = []

for i in range(31):
  _,label_batch=test_generator.next()
  for j in range(label_batch.shape[0]):
    labels.append(label_batch[j])
labels = np.array(labels)
y_labels = np.argmax(labels, axis=1)

### classification accuracy on test set
correct_prediction_count = 0
for i in range(977):
  if y_predict[i] == y_labels[i]:
    correct_prediction_count = correct_prediction_count+1

print("Overall classification accuracy on test set: " + str(correct_prediction_count/977))

Overall classification accuracy on test set: 0.6632548618219037


### Test Teacher

In [ ]:
teacher = Sequential()
resnet_model= tf.keras.applications.ResNet50V2(include_top=False,
                   input_shape=(224,224,3),
                   pooling='avg',classes=2,
                   weights='imagenet')


resnet_model.trainable = True

set_trainable = False
for layer in resnet_model.layers:
    if layer.name in ['conv4_block1_preact_bn', 'conv5_block1_preact_bn']:
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

teacher.add(resnet_model)
teacher.add(Flatten())
teacher.add(Dense(512, activation='relu'))
teacher.add(Dropout(0.3))
teacher.add(Dense(2, activation='softmax'))

teacher.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              metrics=['accuracy'])

teacher.fit_generator(generator=train_generator,
                    steps_per_epoch=train_generator.n // 32,
                    epochs=25,                 
                    shuffle = True,
                    verbose=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  from ipykernel import kernelapp as app


Epoch 1/25
67/67 [==============================] - 39s 503ms/step - loss: 0.4579 - accuracy: 0.7886
Epoch 2/25
67/67 [==============================] - 34s 502ms/step - loss: 0.3111 - accuracy: 0.8698
Epoch 3/25
67/67 [==============================] - 34s 506ms/step - loss: 0.2477 - accuracy: 0.9043
Epoch 4/25
67/67 [==============================] - 34s 502ms/step - loss: 0.1991 - accuracy: 0.9244
Epoch 5/25
67/67 [==============================] - 34s 501ms/step - loss: 0.1644 - accuracy: 0.9263
Epoch 6/25
67/67 [==============================] - 34s 502ms/step - loss: 0.1637 - accuracy: 0.9389
Epoch 7/25
67/67 [==============================] - 34s 500ms/step - loss: 0.1433 - accuracy: 0.9487
Epoch 8/25
67/67 [==============================] - 34s 501ms/step - loss: 0.1308 - accuracy: 0.9515
Epoch 9/25
67/67 [==============================] - 33s 499ms/step - loss: 0.1082 - accuracy: 0.9594
Epoch 10/25
67/67 [==============================] - 34s 500ms/step - loss: 0.0961 - accura

In [ ]:
predict = teacher.predict(test_generator,steps=31)

test_generator.reset()
y_predict = np.argmax(predict, axis=1)
labels = []

for i in range(31):
  _,label_batch=test_generator.next()
  for j in range(label_batch.shape[0]):
    labels.append(label_batch[j])
labels = np.array(labels)
y_labels = np.argmax(labels, axis=1)

### classification accuracy on test set
correct_prediction_count = 0
for i in range(977):
  if y_predict[i] == y_labels[i]:
    correct_prediction_count = correct_prediction_count+1

print("Overall classification accuracy on test set: " + str(correct_prediction_count/977))

Overall classification accuracy on test set: 0.8474923234390993
